In [21]:
import ee
import geemap
import geetools
import rasterio
import folium
from folium import plugins

import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhzwtE2RRdBI01uTETyoCvVMcqDjTayJCWO4YuW_9UhIGjSPCJT5AA

Successfully saved authorization token.


# Set up parameters

In [25]:
data_source = 'LANDSAT/LC08/C01/T1_TOA';
bands = ['B4', 'B3', 'B2'];

wstart_date = '2014-01-01';
wend_date = '2015-01-01';
outfile_name_prefix = 'L8_C01_T1_TOA_' + wstart_date + "_" + wend_date + "_";

# Define the visualization parameters.
vizParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max':[0.2, 0.2, 0.2],
    # 'gamma': [0.95, 1.1, 1]
}

"""
   'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318'
   Create a geometry representing an export region.
""" 
xmin = -122.20;
xmax = -120.45;
ymin = 35.75;
ymax = 37.1;
xmean = (xmin + xmax) / 2.0;
ymean = (ymin + ymax) / 2.0;

# Functions

In [28]:
def add_system_start_time_image(image):
    image = image.addBands(image.metadata('system:time_start').rename("system_start_time"));
    return image;

def add_system_start_time_collection(colss):
    c = colss.map(add_system_start_time_image);
    return c;


In [115]:
def fetch_and_mosaic_IC(data_source, bands, start_date, end_date, AOI):
    an_IC = ee.ImageCollection(data_source)\
              .select(bands)\
              .filterDate(wstart_date, wend_date)\
              .filterBounds(AOI).map(lambda image: image.clip(AOI))
    an_IC = an_IC.sort('system:time_start', true);
    an_IC = add_system_start_time_collection(an_IC);
    start_date_DateType = ee.Date(start_date);
    end_date_DateType = ee.Date(end_date);
    
    # Difference in days between start and end_date
    diff = end_date_DateType.difference(start_date_DateType, 'day');
    
    ## Make a list of all dates
    xrange = ee.List.sequence(0, diff.subtract(1)).map(lambda day: start_date_DateType.advance(day,'day'))
    
    ## Funtion for iteraton over the range of dates
    def day_mosaics(date, newlist):
        ## Cast
        date = ee.Date(date);
        newlist = ee.List(newlist);

        ## Filter an_IC between date and the next day
        filtered = an_IC.filterDate(date, date.advance(1, 'day'));

        image = ee.Image(filtered.mosaic()); ## Make the mosaic

        image = image.copyProperties({source: filtered.first(),
                                      properties: ['system:time_start']
                                      });
        image = image.set({system_time_start: filtered.first().get('system:time_start')});

        ## Add the mosaic to a list only if the an_IC has images
        return ee.List(ee.Algorithms.If(filtered.size(), newlist.add(image), newlist));
    
    ## Iterate over the xrange to make a new list, and then cast the list to an imagecollection
    newcol = ee.ImageCollection(ee.List(xrange.iterate(day_mosaics, ee.List([]))));
    newcol = newcol.sort('system_time_start', true);
    
    return(newcol)

def visualize_and_export_image(image2vis):
    imageRGB_mosaiced1 = image2vis.visualize(vizParams);
    imageRGB_mosaiced1 = imageRGB_mosaiced1.copyProperties({source: image2vis,
                                                          properties: ['system:time_start']
                                                          });
    imageRGB_mosaiced1 = imageRGB_mosaiced1.set({system_time_start: image2vis.get('system_time_start')});
    return imageRGB_mosaiced1;


def add_ee_layer(self, ee_image_object, vis_params, name):
    Atr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>'
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(tiles=map_id_dict['tile_fetcher'].url_format,
                                   attr=Atr,
                                   name=name,
                                   overlay=True,
                                   control=True
                                   ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

In [174]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
                          data = ee_object.getInfo(),
                          name = name,
                          # attr = 'Google Earth Engine',
                          overlay = True,
                          control = True
                          ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [184]:
# # Create a folium map object.
# my_map = folium.Map(location=[xmean, ymean], zoom_start=4, height=500)
# my_map.add_ee_layer(ee_object = big_rect, vis_params = vizParams, name = "big rectangular")
# my_map

folium.GeoJson(data = big_rect.getInfo(),
               name = "fuck me",
               overlay = True,
               control = True)

folium.TileLayer(data = big_rect.getInfo(),
               name = "fuck me",
               overlay = True,
               control = True)

In [189]:
my_map = folium.Map(location=[ymean, xmean], tiles="OpensTreetMap", zoom_start=7, height=500)
my_map.add_ee_layer(ee_object = big_rect, vis_params = vizParams, name = "big rectangular")
my_map

In [192]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
        if ee.image.Image in [type(x) for x in v.values()]:
            folium.TileLayer(
                tiles = v["tile_fetcher"].url_format,
                attr  = 'Google Earth Engine',
                overlay =True,
                name  = k).add_to(mapViz)
        else:
            folium.GeoJson(data = v,
                           name = k).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

In [196]:
buffer = big_rect.buffer(10).getInfo()

# Compute the centroid of the polygon.
centroid = big_rect.centroid().getInfo()

Mapdisplay(center=(ymean, xmean), dicc={'center':centroid,'buffer':buffer}, zoom_start=7)

In [109]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [111]:
# # Import the Image function from the IPython.display module. 
# from IPython.display import Image

# # Display a thumbnail of global elevation.
# Image(url = dem.updateMask(dem.gt(0))
#   .getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 512,
#                 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [159]:
dem = ee.Image('USGS/SRTMGL1_003')

basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}


# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[40.33, -99.42], zoom_start=4, height=500)
my_map


In [ ]:
# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Display ee.Image
dataset = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')
occurrence = dataset.select('occurrence');
occurrenceVis = {'min': 0.0, 'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']}
my_map.add_ee_layer(occurrence, occurrenceVis, 'JRC Surface Water')

# Display ee.Geometry
holePoly = ee.Geometry.Polygon(coords = [[[-35, -10], [-35, 10], [35, 10], [35, -10], [-35, -10]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)
my_map.add_ee_layer(holePoly, {}, 'Polygon')

# Display ee.FeatureCollection
fc = ee.FeatureCollection('TIGER/2018/States')
my_map.add_ee_layer(fc, {}, 'US States')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)